In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd
from nilearn import image as nimg
from nilearn import plotting as nplot
from nilearn import masking as nimask
from nibabel.processing import resample_to_output
from matplotlib import pyplot as plt

In [ ]:
#LOADING IN
#one preprocessed scan
fmri_test_path = 'NYU_0050970_func_preproc.nii'
#nibable image

fmri_image_path = 'C:/Users/ahmm9/OneDrive - ITU/ITU/Research_proj/prepros_data/Outputs/ccs/filt_global/func_preproc'
#fmri_image_path = '../DATA/pre_proc_ABIDE1/func_preproc'
img = nib.load(fmri_test_path).get_fdata()

print(img.shape)
print(f"The .nii files are stored in memory as numpy's: {type(img)}.")


#parcellation mask
mask_path = 'Yeo2011_7Networks_MNI152_FreeSurferConformed1mm_LiberalMask.nii'
img2 = nib.load(mask_path).get_fdata()
img2.shape


Plot data and mask

In [ ]:
plt.style.use('default')
fig, axes = plt.subplots(4, 10, figsize=(12, 4))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(img[:, :, i, 0])
plt.show()

In [ ]:
plt.style.use('default')
fig, axes = plt.subplots(4, 12, figsize=(12, 4))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(img2[:, :, 125 + i, 0])  
    # Start from the 31st image (index 30)#TODO still this with different indexing??
plt.show()


In [ ]:
# #separate mask values #OLD
# def create_masks(dataset):
#     masks = {}
#     for value in range(1, 8):  # Assuming values are between 1 and 7
#         masks[value] = dataset == value
#     return masks

# masks = create_masks(img2)

# print(masks[1].shape)

In [ ]:
# plt.style.use('default')
# fig, axes = plt.subplots(32, 4, figsize=(10, 10))
# for i, ax in enumerate(axes.reshape(-1)):
#     ax.imshow(masks[1][:, :, i, 0])  
# plt.show()

In [ ]:
#load with NILearn
ni_image = nimg.load_img(fmri_test_path)
ni_mask = nimg.load_img(mask_path)
print(ni_image.shape)
print(ni_mask.shape)

In [ ]:
ni_image.affine

In [ ]:
nplot.plot_roi(ni_mask, ni_image.slicer[:,:,:,54])
print(ni_mask.shape)
print(ni_image.shape)

In [ ]:
#nilearn reshape dims of mask to match
reshaped_mask = nimg.resample_img(ni_mask,target_affine=ni_image.affine, interpolation='nearest',target_shape=ni_image.shape[:3])
reshaped_mask.shape

In [ ]:
#plot the mask onto one slice of the frmi image
nplot.plot_roi(reshaped_mask, ni_image.slicer[:,:,:,5])

In [ ]:
#separate the regions of the mask
def create_parcellation_masks(nifti_mask, nifti_image):
    # Load the data from the Nifti image
    data = nifti_mask.get_fdata().astype(int)
    
    # Create a dictionary to store each mask
    masks = {}
    for value in range(1, 8):  # Assuming values are between 1 and 7
        masks[value] = nib.Nifti1Image((data == value).astype(float), nifti_image.affine) # Create a mask for each unique value
    return masks

mask_dict = create_parcellation_masks(reshaped_mask,ni_image)
print(type(mask_dict[1]))
print(mask_dict[1].shape)

In [ ]:
mask_reasonable_shape = reshaped_mask.get_fdata().astype(int)[:,:,:,0]
not_brain = (mask_reasonable_shape == 0).sum()
brain = (mask_reasonable_shape != 0).sum()
print(brain,not_brain) # not brain is relative to parcellation
print((brain/(brain+not_brain))*100,"%")

In [ ]:
ni_image.shape

In [ ]:
#apply mask
output = {}
means_no_threshold = []
voxelsum = 0
for i in range(1,8):
    output[i] = nimask.apply_mask(ni_image, mask_dict[i])
    nplot.plot_roi(mask_dict[i], ni_image.slicer[:,:,:,54])
    nplot.show()
    voxelsum += np.sum(output[i])
    means_no_threshold.append(np.mean(output[i], axis = 1))
    
print(means_no_threshold)
print(means_no_threshold[1].shape)
print(brain)
print(voxelsum)
threshold = voxelsum/brain
print(threshold)

In [ ]:
def apply_mask(image,mask):
    output_dict = {}
    for i in range(1,8):
        stack = []
        masked = nimask.apply_mask(image, mask[i])
        row_check = masked[0][masked[0] != 0]
        for row in masked:
            masked_no_zeros = row[row != 0]
            assert(len(masked_no_zeros) == len(row_check))
            stack.append(masked_no_zeros)
        output_dict[i] = np.array(stack)
    return output_dict

output2 = apply_mask(ni_image,mask_dict)


        

In [ ]:
some = output2[1][1][output2[1][1] != 0]
some2 = output2[1][1][output2[1][1] == 0]

# for i in range(1,175):
#     print((output2[5][i][output2[5][i] != 0]).shape)
print(some.shape)
print(some2.shape)
print(output2[1][1].shape)
print(output2[1][1].shape[0]- some.shape[0])

In [ ]:
print(ni_image.shape)
print(len(mask_dict))
print(len(output2))
print(output2[5].shape)
print("here",np.sum(output2[5]==0.0,axis=1))
print("here",output2[5]==0)

print(np.array_equal(output[1], output2[1]))
print(mask_dict[5].shape)


In [ ]:
for i in range (1,8):
    print("here", np.max(np.sum(output2[i] == 0.0, axis=1)))



In [ ]:
import os

def compute_voxel_count_total(data):
    voxel_count = 0
    for key in data.keys():
        voxel_count += len(data[key][0])
    return voxel_count

def compute_voxel_count_per_region(data,t = 0):
    voxel_count = 0
    for i in range(len(data.keys())):
        voxel_count += len(data[i+1][t])
    return voxel_count
print(compute_voxel_count_per_region(output))


In [ ]:
#get all images into list #TODO get from fmri_image_path
all_images = []
seg = pd.read_csv("phenotypic_NYU.csv")
labels = []
for filename in os.listdir(fmri_image_path):
    if filename.endswith('.nii') or filename.endswith('.nii.gz'):
        file_path = os.path.join(fmri_image_path, filename)
        labels.append(int(seg[seg["SUB_ID"]==int(filename[6:11])]["DX_GROUP"]))
        img = nimg.load_img(file_path)
        all_images.append(img)
print(np.array(all_images).shape)
print(np.array(labels).shape)

#1:30 runtime, use paths instead if only running some


In [ ]:
#check affine is same for all images
print(len(all_images))
prev_img = all_images[0]
for img in all_images:
    if (prev_img.affine != img.affine).all():
        print("different")
    prev_img = img

In [ ]:
import pickle

In [ ]:
# threshold finder
# all group region all time
# all region averages, all participants

In [ ]:
print(np.array(all_images).shape)
print(all_images[0])

In [ ]:
all_masked_images = []
start_index = 0
# running mask apply, save every 10th iteration, can be restarted and interrupted
try:
    with open('masked_image.pkl', 'rb') as f:
        print(f)
        all_masked_images = pickle.load(f)
        start_index = len(all_masked_images)
        print(f"Resuming from index {start_index}")
except FileNotFoundError:
    print("No previous save found, starting from scratch")
    
for i in range(start_index,len(all_images)):
    all_masked_images.append(apply_mask(all_images[i],mask_dict))
    if (i % 10 == 0):
        with open('masked_image.pkl', 'wb') as f:
            pickle.dump(all_masked_images,f)
        print("saved at index", i)

print("Applied mask to all images")

print(np.array(all_masked_images[0]).shape)
print(len(all_images) == len(all_masked_images))
print(all_images[0].shape[3] == len(all_masked_images[0][1]))

In [ ]:
print(all_masked_images[0][5])

In [ ]:
#if timesteps isnt the same the calculaitons need to be weighted
# this bit checks all scans are 175 timesteps long
for i in range(len(all_images)):
    if all_images[i].shape[3] != 175:
        print( all_images[i] )

In [ ]:
def threshold_per_timestep(data,time_steps):
    """each participant, each timestep"""
    thresholds = np.zeros((len(data),time_steps))
    for i in range(len(data)):
        for t in range(time_steps):
            voxel_sum = 0
            voxel_count = 0
            for k in range(len(data[i].keys())):
                voxel_sum += np.sum(data[i][k+1][t]) 
                print("data:", data[i][k+1][t])
                print("sum: ", np.sum(data[i][k+1][t]) )
                voxel_count += data[i][k+1].shape[1]
            print("voxelsum:", voxelsum)
            print("voxel_ count true:" , voxel_count)
            print("fdsopf", data[i][k+1].shape)
            means = voxel_sum/voxel_count
            print("means:", means)
            print("true:", data[i][1])
            thresholds[i][t] = means
    return thresholds

#Bit of a shit way to do it. Just take average within the bin instead
def threshold_per_bin(data,bin_size, time_steps):
    """each participant, each bin (div by 5 is easiest)"""
    time_steps = int(time_steps/bin_size)
    thresholds = np.zeros((len(data),time_steps,7))
    for i in range(len(data)):
        for t in range(time_steps):
            voxel_sum = np.zeros(7)
            index_offset = t * bin_size
            voxel_count = 0
            for b in range(bin_size):
                voxel_count += compute_voxel_count_per_region(data[i],index_offset+b)  
                voxel_sum_temp = np.zeros(7)
                for k in range(len(data[i].keys())):
                    voxel_sum_temp[k] = np.sum(data[i][k+1][index_offset+b])  
                voxel_sum += voxel_sum_temp
            means = voxel_sum/voxel_count
            thresholds[i][t] = means
    return thresholds

def threshold_per_participant(data,time_steps):
    """each participant, whole timeframe"""
    thresholds = np.zeros((len(data),7))
    for i in range(len(data)):
        voxel_sum = np.empty(7)
        voxel_count = compute_voxel_count_per_region(data[i],0) * time_steps 
        for k in range(len(data[i].keys())):
            voxel_sum[k] = np.sum(data[i][k+1])
        means = voxel_sum/voxel_count
        thresholds[i] = means
    return thresholds

# def threshold_participant_region_all_time(data):
#     all_thesholds = []
#     for participant in range(len(data)):
#         print(participant)
#         l_thresholds = [0 for i in range(len(data[0].keys()))] # set to 0 for all regions
#         for i in range(len(l_thresholds)):
#             # print(f"{i}:")
#             # voxels in the mask by number of timesteps by number of participants in group
#             voxel_count = len(data[0][i+1][0]) * len(data[0][i+1]) * sum(labels == l)

#             voxel_sum = 0
#             for participant in range(len(data)):
#                 if labels[participant] == l: # only within group

#                 #print(len(data[participant][i+1]))
#                 #print(np.sum(data[participant][i+1]))
#                     voxel_sum += np.sum(data[participant][i+1])#[participant][region]
#             l_thresholds[i] = voxel_sum/voxel_count
#             all_thresholds.append(l_thresholds)
#     return all_thresholds


def threshold_groups_region_all_time(data,labels):
    """Add docu to functions. Also make more functions, fuck the free line scripting"""
    labels = np.array(labels)
    all_thresholds = dict()
    for l in set(labels):
        l_thresholds = [0 for i in range(len(data[0].keys()))] # set to 0 for all regions
        for i in range(len(l_thresholds)):
            # print(f"{i}:")
            # voxels in the mask by number of timesteps by number of participants in group
            voxel_count = len(data[0][i+1][0]) * len(data[0][i+1]) * sum(labels == l)

            voxel_sum = 0
            for participant in range(len(data)):
                if labels[participant] == l: # only within group

                #print(len(data[participant][i+1]))
                #print(np.sum(data[participant][i+1]))
                    voxel_sum += np.sum(data[participant][i+1])#[participant][region]
            l_thresholds[i] = voxel_sum/voxel_count
        all_thresholds[l]=l_thresholds
    return all_thresholds


In [ ]:
#1  is autism 2 is TD/control
#threshold_rat = threshold_participant_region_all_time(all_masked_images)
threshold_grat = threshold_groups_region_all_time(all_masked_images,labels)
threshold_grat
#threshold_rat

In [ ]:
threshold_timestep = threshold_per_timestep(all_masked_images,175)
print("timestep: ", threshold_timestep)
    

group_threshold_timestep = threshold_timestep
print(len(group_threshold_timestep))
print(len(group_threshold_timestep[1]))
print(len(group_threshold_timestep[2]))
print("GTT_SHAPE:", group_threshold_timestep[0].shape)

GTT_SHAPE = group_threshold_timestep[0].shape

bin_size = 5
threshold_bin = threshold_per_bin(all_masked_images,bin_size,175)
print("bin: ", threshold_bin.shape)

group_threshold_bin = np.transpose(threshold_bin,(0,2,1))
print(len(group_threshold_bin))
print(len(group_threshold_bin[1]))
print(len(group_threshold_bin[2]))
print("GTB_SHAPE: ",group_threshold_bin[0].shape)

GTB_SHAPE = group_threshold_bin[0].shape

threshold_participant = threshold_per_participant(all_masked_images,175)
print("participant: ", threshold_participant.shape)

group_threshold_participant= np.transpose(threshold_participant,(1,0))
print(len(group_threshold_participant))
print(len(group_threshold_participant[1]))
print(len(group_threshold_participant[2]))
print("GTP_SHAPE: ", group_threshold_participant[0].shape)

GTP_SHAPE = group_threshold_participant[0].shape


In [ ]:
len(all_masked_images[0].keys())

In [ ]:
# def binarise(data,labels,thresholds):
#     labels = np.array(labels)
#     for_eda_by_group = {1:[],2:[]}
#     for scan_num,one_scan in enumerate(data):
#         image_label = labels[scan_num]
#         one_participant_bin = np.array(pd.DataFrame.from_dict(binarize_participant (one_scan,thresholds[image_label]),orient="index"))
#         for_eda_by_group[image_label].append(one_participant_bin)
#     for group in for_eda_by_group:
#         for_eda_by_group[group] = np.concatenate(for_eda_by_group[group],axis=1,dtype=int)
#     return for_eda_by_group
# 
# def binarize_participant(participant,thresholds):
#     means_with_threshold = dict()
#     for i in participant.keys():
#         mean_image = (np.mean(participant[i], axis = 1))-thresholds[i-1]
#         bin_image = np.where(mean_image>0, 1,-1)
#         means_with_threshold[i] = bin_image
#     return means_with_threshold

In [675]:
## New version
def binarise(data,thresholds):
    for_eda_by_group = {1:[],2:[]}
    for scan_num,one_scan in enumerate(data):
        if (scan_num == 0):
            image_label = labels[scan_num]
            one_participant_bin = np.array(pd.DataFrame.from_dict(apply_threshold(one_scan,thresholds[scan_num]),orient="index"))
            for_eda_by_group[image_label].append(one_participant_bin)
    for group in for_eda_by_group:
        if (len(for_eda_by_group[group]) > 0):
            for_eda_by_group[group] = np.concatenate(for_eda_by_group[group],axis=1,dtype=int)
    return for_eda_by_group

def apply_threshold(participant, thresholds):
    binarized_participant = {}
    if thresholds.shape == GTT_SHAPE:
        i = 0
        for k in participant.keys():
            
            means = (np.mean(participant[k], axis = 1))-thresholds
            if (i == 0):
                print("thresh:", thresholds, " mean:", np.mean(participant[k], axis = 1), " result: ", means)
                print("means shape: ", (np.mean(participant[k], axis = 0)).shape)
            binarized_participant[k] = np.where(means>0, 1,-1)   
        i = 1
    elif thresholds.shape == GTB_SHAPE:
        for k in participant.keys():
            expanded_thresholds = np.repeat(thresholds, 5)
            binarized_participant[k] = np.where(participant[k] > expanded_thresholds, 1, -1)
    elif thresholds.shape == GTP_SHAPE:
        for k in participant.keys():
            binarized_participant[k] = np.where(participant[k] > thresholds[k], 1, -1)
    else:
        raise ValueError("thresholds does have correct dimensions, found dimension:", thresholds.shape )
    return binarized_participant
    

# def binarize_participant(participant,thresholds):
#     new_participant = participant.copy()
#     means_with_threshold = dict()
#     for k in participant.keys():
#         for t in range(len(participant[k])):
#             participant[k][i]
#         
#         mean_image = (np.mean(participant[i], axis = 1))-thresholds[i-1]
#         bin_image = np.where(mean_image>0, 1,-1)
#         means_with_threshold[i] = bin_image
#     return means_with_threshold

In [676]:
holder = np.array(all_masked_images)
print(holder.shape)
print(len(holder[0]))
print(len(holder[0][1]))
print(len(holder[0][5][0]))

(171,)
7
175
2780


In [677]:
binarized = binarise(all_masked_images,group_threshold_timestep)
binarized[1].shape

thresh: [10217.66874509 10217.65229238 10217.29499607 10215.94419701
 10214.20728321 10213.30500393 10214.28134661 10217.00513492
 10219.84469479 10220.68357349 10218.71346607 10215.36421273
 10213.20026199 10213.60015719 10215.49321457 10216.55100865
 10215.64825779 10213.98718889 10213.69310977 10215.49051611
 10218.03730679 10219.51600734 10219.52433849 10219.10523972
 10219.11427823 10219.07443018 10217.93759497 10215.74875557
 10214.06963584 10214.55357611 10217.24335866 10220.44878177
 10222.23764737 10221.90078596 10220.1018863  10218.00683783
 10216.52135185 10216.10518732 10216.74770762 10217.80681163
 10218.11417343 10216.73631124 10213.91275871 10211.12124705
 10209.99795651 10211.04757663 10213.36426513 10215.50725701
 10216.56866649 10216.54776002 10216.04419701 10215.80479434
 10216.38375688 10217.82481006 10219.47374902 10220.30830495
 10219.77516374 10218.3573487  10217.10903851 10216.53869531
 10216.12439088 10215.12837307 10213.76340058 10213.17631648
 10214.11270631 

(7, 175)

In [678]:
#save
for i in binarized.keys():
    np.savetxt(f"Binarized_Data_Group_{i}",binarized[i],fmt="%d",delimiter=",")

In [ ]:
#TODO:
# binarise
# is threshold different by group??? and perticipant?? and variance
# save the data, separate data by groups (structure)
# threshold by region
# sliding timeframes
# binsize
# concatenation of groups
# writing
# formatting of notebook


In [ ]:
# mean_activations = np.array(means)
# mean_activations.shape
# threshold